In [ ]:
# OPTIONAL: Notebook header / notes. Not required for execution — informational only.
# User asks a question
# agents accepts the question / agent does a lookup on the user / regular user or rls user
# if regular user then no RLS applied otherwise if rls user then Country='Canada' applied
# generates a DAX statement with rls or with regular
# DAX statement is issued to PBI Semantic Model


<img src="./3 Steps - Enhance Your Power BI Embed Solution - Custom Copilot Architecture.png" alt="Description" style="max-width:100%; height:auto;">

# 🤖 Semantic Kernel Agentic DAX Copilot

This notebook demonstrates an **agentic approach** to natural language → DAX query generation using:
- **Semantic Kernel** for agent orchestration
- **Plugin-based architecture** for modularity  
- **Service principal authentication** for Power BI access
- **Row-level security (RLS)** support

## Architecture Overview:
```
🤖 DAX Copilot Agent
├── 👤 UserContextPlugin → Determines RLS context
├── 🧠 DAXGenerationPlugin → Natural language → DAX
├── 🔐 AuthenticationPlugin → Service principal tokens  
└── 📊 QueryExecutionPlugin → Execute DAX → DataFrame
```

In [ ]:
# 📦 Install Semantic Kernel and Dependencies
%pip install semantic-kernel --quiet
%pip install msal --quiet  
%pip install pandas --quiet
%pip install pythonnet --quiet

print("✅ Semantic Kernel and dependencies installed!")

In [ ]:
# 🔧 Configuration and Core Imports
import asyncio
import os
from typing import Annotated

# Semantic Kernel imports
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.kernel import Kernel

# Data and auth imports
import pandas as pd
import msal
import base64
import json
import time
from datetime import datetime

print("✅ Core imports loaded!")

# Configuration
AZURE_OPENAI_API_KEY = "youraoaikey"
AZURE_OPENAI_ENDPOINT = "youraoaiendpoint"
AZURE_OPENAI_DEPLOYMENT = "youraoaideployment"
API_VERSION = "youraoaiapiversion"

# Service Principal Configuration
CLIENT_ID = "yourserviceprincipalclientid"
CLIENT_SECRET = "yourserviceprincipalclientsecret"
TENANT_ID = "yourserviceprincipaltenantid"

# Power BI Configuration
POWERBI_WORKSPACE = "yourpowerbiworkspacename"
SEMANTIC_MODEL = "yoursemanticmodelname"

print("✅ Configuration loaded!")

In [3]:
# 👤 UserContext Plugin - Determines RLS Context
class UserContextPlugin:
    """Plugin to determine user context and RLS requirements"""
    
    @kernel_function(
        description="Determines if a user requires row-level security filtering",
        name="get_user_rls_context",
    )
    def get_user_rls_context(
        self, 
        user_type: Annotated[str, "Type of user: 'rls' for restricted, 'regular' for unrestricted"] = "regular"
    ) -> str:
        """
        Determines the RLS context for a user.
        Returns JSON string with user context information.
        """
        if user_type.lower() == 'rls':
            context = {
                'user': 'user1@customer.com',
                'user_type': 'rls',
                'rls_filter': 'DimSalesTerritory[Sales Territory Country] = "Canada"',
                'rls_instruction': 'Row-level security: Always apply the following DAX filter in your query: FILTER(DimSalesTerritory, DimSalesTerritory[Sales Territory Country] = "Canada")'
            }
        else:
            context = {
                'user': 'regular_user@customer.com', 
                'user_type': 'regular',
                'rls_filter': None,
                'rls_instruction': ''
            }
        
        return json.dumps(context, indent=2)

# Test the plugin
user_plugin = UserContextPlugin()
print("🧪 Testing UserContext Plugin:")
print("Regular user:", user_plugin.get_user_rls_context("regular"))
print("\nRLS user:", user_plugin.get_user_rls_context("rls"))

🧪 Testing UserContext Plugin:
Regular user: {
  "user": "regular_user@customer.com",
  "user_type": "regular",
  "rls_filter": null,
  "rls_instruction": ""
}

RLS user: {
  "user": "user1@customer.com",
  "user_type": "rls",
  "rls_filter": "DimSalesTerritory[Sales Territory Country] = \"Canada\"",
  "rls_instruction": "Row-level security: Always apply the following DAX filter in your query: FILTER(DimSalesTerritory, DimSalesTerritory[Sales Territory Country] = \"Canada\")"
}


In [ ]:
# 🔐 AuthenticationPlugin - Service Principal Token Management
class AuthenticationPlugin:
    """Plugin to handle service principal authentication for Power BI"""
    
    def __init__(self):
        self.client_id = CLIENT_ID
        self.client_secret = CLIENT_SECRET
        self.tenant_id = TENANT_ID
        self.authority = f"https://login.microsoftonline.com/{self.tenant_id}"
        
    @kernel_function(
        description="Acquires Azure AD access token for Power BI using service principal",
        name="get_powerbi_token",
    )
    def get_powerbi_token(self) -> str:
        """
        Acquires access token for Power BI API using service principal credentials
        Returns: JSON string with token information
        """
        try:
            app = msal.ConfidentialClientApplication(
                self.client_id,
                client_credential=self.client_secret,
                authority=self.authority
            )
            
            token_result = app.acquire_token_for_client(
                scopes=["https://analysis.windows.net/powerbi/api/.default"]
            )
            
            if 'access_token' not in token_result:
                raise RuntimeError(f"Failed to acquire token: {token_result}")
            
            # Parse token expiry
            access_token = token_result['access_token']
            try:
                payload_b64 = access_token.split('.')[1]
                payload_b64 += '=' * (-len(payload_b64) % 4)
                payload = json.loads(base64.urlsafe_b64decode(payload_b64).decode('utf8'))
                exp = int(payload.get('exp'))
            except Exception:
                exp = int(time.time()) + int(token_result.get('expires_in', 3600))
            
            result = {
                'access_token': access_token,
                'expires_at': exp,
                'token_type': token_result.get('token_type', 'Bearer'),
                'status': 'success'
            }
            
            return json.dumps(result, indent=2)
            
        except Exception as e:
            error_result = {
                'status': 'error',
                'error_message': str(e),
                'access_token': None
            }
            return json.dumps(error_result, indent=2)

# Test the AuthenticationPlugin
auth_plugin = AuthenticationPlugin()
print("🧪 Testing AuthenticationPlugin:")
token_result = auth_plugin.get_powerbi_token()
print("Token acquired:", "✅" if "success" in token_result else "❌")
if "success" in token_result:
    print("Token preview (first 50 chars):", json.loads(token_result)['access_token'][:50] + "...")

In [ ]:
# 🚀 Enhanced DAX Generation Plugin - Production Ready with Real LLM
from openai import AzureOpenAI

class EnhancedDAXGenerationPlugin:
    """Production-ready DAX Generation Plugin with real Azure OpenAI integration"""
    
    def __init__(self, azure_openai_client):
        self.client = azure_openai_client
        self.semantic_model_metadata = """
Semantic model metadata:
- Table: FactInternetSales
    • FactInternetSales[SalesAmount] (decimal)
    • FactInternetSales[Transaction Count] (decimal) 
    • FactInternetSales[OrderDate] (date)
    • FactInternetSales table contains a measure 'Revenue'=Sum(FactInternetSales[SalesAmount]) and another measure called 'Sales'='Revenue'=Sum(FactInternetSales[SalesAmount])
- Table: DimSalesTerritory
    • DimSalesTerritory[SalesTerritoryKey] (int)
    • DimSalesTerritory[Sales Territory Region] (string) 
    • DimSalesTerritory[Sales Territory Country] (string)
- Table: DimDate
    • DimDate[DateKey] (int)
    • DimDate[CalendarYear] (int)

Relationships:
- FactInternetSales[SalesTerritoryKey] → DimSalesTerritory[SalesTerritoryKey]
- FactInternetSales[DateKey] → DimDate[DateKey]
"""
    
    @kernel_function(
        description="Generates DAX query from natural language using Azure OpenAI with RLS support",
        name="generate_dax_query_llm",
    )
    def generate_dax_query_llm(
        self,
        question: Annotated[str, "Natural language question about data"],
        user_context: Annotated[str, "JSON string with user context including RLS requirements"] = "{}"
    ) -> str:
        """
        Generates a DAX query from natural language using Azure OpenAI LLM
        Handles RLS filtering automatically based on user context
        """
        try:
            # Parse user context
            context = json.loads(user_context)
            rls_instruction = context.get('rls_instruction', '')
            
            # Build system message with RLS context
            system_message = f'''You are a Power BI DAX expert. Generate only the DAX query.
Do not include explanations, markdown, triple backticks, or language tags.

{self.semantic_model_metadata.strip()}
{rls_instruction}'''

            # Call Azure OpenAI with structured prompt
            response = self.client.chat.completions.create(
                model=AZURE_OPENAI_DEPLOYMENT,
                messages=[
                    {"role": "system", "content": system_message.strip()},
                    {"role": "user", "content": question}
                ],
                temperature=0
            )
            
            # Clean and return the DAX query
            import re
            dax_query = re.sub(
                r"^```[^\n]*\n|\n```$", 
                "", 
                response.choices[0].message.content, 
                flags=re.MULTILINE
            ).strip()
            
            return dax_query
            
        except Exception as e:
            return f"Error generating DAX query: {str(e)}"

# Initialize Azure OpenAI client for production plugin
enhanced_openai_client = AzureOpenAI(
    api_version=API_VERSION,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# Create the production-ready enhanced plugin
enhanced_dax_plugin = EnhancedDAXGenerationPlugin(enhanced_openai_client)
print("✅ Enhanced DAX Generation Plugin created with real Azure OpenAI integration!")
print("🧠 This plugin encapsulates LLM calls within the agentic architecture")

In [ ]:
# 🏭 Production Kernel & Agent - Complete 4-Plugin Architecture

async def initialize_production_kernel():
    """Initialize Production Kernel with all 4 plugins for complete workflow"""
    kernel = Kernel()
    
    # Add Azure OpenAI service
    kernel.add_service(
        AzureChatCompletion(
            deployment_name=AZURE_OPENAI_DEPLOYMENT,
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=API_VERSION,
            service_id="aoai"
        )
    )
    
    # Add all production plugins
    kernel.add_plugin(UserContextPlugin(), plugin_name="UserContext")
    kernel.add_plugin(enhanced_dax_plugin, plugin_name="DAXGeneration") 
    kernel.add_plugin(AuthenticationPlugin(), plugin_name="Authentication")
    kernel.add_plugin(QueryExecutionPlugin(), plugin_name="QueryExecution")
    
    print("🏭 Production Kernel initialized with:")
    print("  🌐 Azure OpenAI service")
    print("  👤 UserContext plugin")
    print("  🧠 Enhanced DAX Generation plugin (real LLM)")
    print("  🔐 Authentication plugin")
    print("  📊 Query Execution plugin")
    
    return kernel

# Production Agent - Complete End-to-End Workflow
async def production_dax_copilot_agent(question: str, user_type: str = "regular"):
    """
    🏭 PRODUCTION AGENT: Complete agentic workflow
    Context → LLM → Authentication → Query Execution → Results
    """
    print(f"🏭 Production DAX Copilot Agent Processing: '{question}'")
    print(f"👤 User Type: {user_type}")
    
    try:
        # Step 1: Determine user context and RLS requirements
        print("\n📋 Step 1: Determining user context...")
        user_context = await production_kernel.invoke(
            function_name="get_user_rls_context",
            plugin_name="UserContext", 
            user_type=user_type
        )
        print("✅ User context determined")
        
        # Step 2: Generate DAX query with real Azure OpenAI
        print("\n🧠 Step 2: Generating DAX query with Azure OpenAI...")
        dax_query = await production_kernel.invoke(
            function_name="generate_dax_query_llm",
            plugin_name="DAXGeneration",
            question=question,
            user_context=user_context.value
        )
        print("✅ DAX query generated with real LLM")
        print(f"Generated DAX (preview): {dax_query.value[:80]}...")
        
        # Step 3: Acquire Power BI access token
        print("\n🔐 Step 3: Acquiring Power BI access token...")
        token_result = await production_kernel.invoke(
            function_name="get_powerbi_token",
            plugin_name="Authentication"
        )
        print("✅ Access token acquired")
        
        # Step 4: Execute DAX query against Power BI
        print("\n📊 Step 4: Executing DAX query against Power BI...")
        query_result = await production_kernel.invoke(
            function_name="execute_dax_query",
            plugin_name="QueryExecution",
            dax_query=dax_query.value,
            access_token=token_result.value
        )
        print("✅ DAX query executed")
        
        # Parse and return complete results
        result_data = json.loads(query_result.value)
        
        return {
            'question': question,
            'user_context': json.loads(user_context.value),
            'dax_query': dax_query.value,
            'execution_result': result_data,
            'workflow_complete': True,
            'agent_type': 'production'
        }
        
    except Exception as e:
        return {
            'question': question,
            'error': str(e),
            'workflow_complete': False,
            'agent_type': 'production'
        }

# Initialize the production kernel
production_kernel = await initialize_production_kernel()
print("\n🚀 Production system ready!")
print("Use: await production_dax_copilot_agent('your question', 'regular|rls')")

In [9]:
# 📊 QueryExecutionPlugin - Execute DAX Against Power BI
class QueryExecutionPlugin:
    """Plugin to execute DAX queries against Power BI semantic model"""
    
    def __init__(self):
        self.workspace = POWERBI_WORKSPACE
        self.semantic_model = SEMANTIC_MODEL
        
    @kernel_function(
        description="Executes a DAX query against Power BI semantic model and returns DataFrame",
        name="execute_dax_query",
    )
    def execute_dax_query(
        self,
        dax_query: Annotated[str, "The DAX query to execute"],
        access_token: Annotated[str, "JSON string containing access token information"]
    ) -> str:
        """
        Executes DAX query against Power BI semantic model using ADOMD.NET
        Returns: JSON string with query results or error information
        """
        try:
            # Parse access token
            token_info = json.loads(access_token)
            if token_info.get('status') != 'success':
                return json.dumps({
                    'status': 'error',
                    'error': 'Invalid access token',
                    'data': None
                })
            
            # Setup ADOMD.NET
            import clr
            import sys
            sys.path.append(r'C:\Program Files\Microsoft.NET\ADOMD.NET\160')
            clr.AddReference('Microsoft.AnalysisServices.AdomdClient')
            
            from Microsoft.AnalysisServices.AdomdClient import AdomdConnection, AdomdCommand, AccessToken as AdomdAccessToken
            from System import DateTimeOffset
            
            # Create CLR access token
            access_token_str = token_info['access_token']
            expiry = DateTimeOffset.FromUnixTimeSeconds(token_info['expires_at'])
            token_obj = AdomdAccessToken(access_token_str, expiry, None)
            
            # Build connection string
            connection_string = (
                f"Provider=MSOLAP;"
                f"Data Source=powerbi://api.powerbi.com/v1.0/myorg/{self.workspace};"
                f"Initial Catalog={self.semantic_model};"
            )
            
            # Execute query
            conn = AdomdConnection(connection_string)
            conn.AccessToken = token_obj
            conn.Open()
            
            try:
                cmd = AdomdCommand(dax_query, conn)
                reader = cmd.ExecuteReader()
                
                # Get column names
                columns = [reader.GetName(i) for i in range(reader.FieldCount)]
                
                # Read all rows
                rows = []
                while reader.Read():
                    row = []
                    for i in range(reader.FieldCount):
                        val = reader.GetValue(i)
                        if 'System.Decimal' in str(type(val)):
                            val = float(val.ToString())
                        row.append(val)
                    rows.append(row)
                
                reader.Close()
                
            finally:
                conn.Close()
            
            # Return results as JSON
            result = {
                'status': 'success',
                'columns': columns,
                'rows': rows,
                'row_count': len(rows),
                'executed_query': dax_query
            }
            
            return json.dumps(result, indent=2)
            
        except Exception as e:
            error_result = {
                'status': 'error',
                'error_message': str(e),
                'data': None,
                'attempted_query': dax_query
            }
            return json.dumps(error_result, indent=2)

# Test the QueryExecutionPlugin (will need valid token and DAX query)
query_plugin = QueryExecutionPlugin()
print("✅ QueryExecutionPlugin created successfully!")
print(f"📊 Configured for: {query_plugin.workspace}/{query_plugin.semantic_model}")

✅ QueryExecutionPlugin created successfully!
📊 Configured for: embed-customcopilot/adventuerworks2017


In [18]:
# 🧪 Test Complete Agentic Workflow - Regular User
print("=" * 60)
print("🧪 TESTING COMPLETE AGENTIC WORKFLOW - REGULAR USER")
print("=" * 60)

regular_result = await complete_dax_copilot_agent("Show Sales by Year", "regular")

if regular_result['workflow_complete']:
    print(f"\n📊 REGULAR USER RESULTS:")
    print(f"Question: {regular_result['question']}")
    print(f"User Type: {regular_result['user_context']['user_type']}")
    print(f"RLS Applied: {'Yes' if regular_result['user_context']['rls_filter'] else 'No'}")
    print(f"\n📜 COMPLETE Generated DAX Query:")
    print("=" * 50)
    print(regular_result['dax_query'])
    print("=" * 50)
    
    exec_result = regular_result['execution_result']
    if exec_result['status'] == 'success':
        print(f"\n📈 Query Results:")
        print(f"Columns: {exec_result['columns']}")
        print(f"Row Count: {exec_result['row_count']}")
        print("✅ Regular user workflow completed successfully!")
        
        # Display ALL rows for complete comparison
        if exec_result['rows']:
            print(f"\n📊 COMPLETE Data Results (all {exec_result['row_count']} rows):")
            for i, row in enumerate(exec_result['rows']):
                print(f"Row {i+1}: {row}")
    else:
        print(f"❌ Query execution failed: {exec_result['error_message']}")
else:
    print(f"❌ Workflow failed: {regular_result.get('error', 'Unknown error')}")

print("\n" + "=" * 60)

🧪 TESTING COMPLETE AGENTIC WORKFLOW - REGULAR USER
🤖 Complete DAX Copilot Agent Processing: 'Show Sales by Year'
👤 User Type: regular

📋 Step 1: Determining user context...
✅ User context determined

🧠 Step 2: Generating DAX query with LLM...
✅ DAX query generated with LLM
Generated DAX: EVALUATE
SUMMARIZECOLUMNS(
    DimDate[CalendarYear],
    "Sales", [Sales]
)...

🔐 Step 3: Acquiring Power BI access token...
✅ Access token acquired

📊 Step 4: Executing DAX query against Power BI...
✅ DAX query executed

📊 REGULAR USER RESULTS:
Question: Show Sales by Year
User Type: regular
RLS Applied: No

📜 COMPLETE Generated DAX Query:
EVALUATE
SUMMARIZECOLUMNS(
    DimDate[CalendarYear],
    "Sales", [Sales]
)

📈 Query Results:
Columns: ['DimDate[CalendarYear]', '[Sales]']
Row Count: 4
✅ Regular user workflow completed successfully!

📊 COMPLETE Data Results (all 4 rows):
Row 1: [2011, 6852489.3846]
Row 2: [2012, 5836345.8175]
Row 3: [2013, 16044747.2986]
Row 4: [2014, 625094.72]



In [19]:
# 🧪 Test Complete Agentic Workflow - RLS User
print("=" * 60)
print("🧪 TESTING COMPLETE AGENTIC WORKFLOW - RLS USER")
print("=" * 60)

rls_result = await complete_dax_copilot_agent("Show Sales by Year", "rls")

if rls_result['workflow_complete']:
    print(f"\n📊 RLS USER RESULTS:")
    print(f"Question: {rls_result['question']}")
    print(f"User Type: {rls_result['user_context']['user_type']}")
    print(f"RLS Applied: {'Yes' if rls_result['user_context']['rls_filter'] else 'No'}")
    if rls_result['user_context']['rls_filter']:
        print(f"RLS Filter Applied: {rls_result['user_context']['rls_filter']}")
    
    print(f"\n📜 COMPLETE Generated DAX Query (with RLS filter):")
    print("=" * 50)
    print(rls_result['dax_query'])
    print("=" * 50)
    
    exec_result = rls_result['execution_result']
    if exec_result['status'] == 'success':
        print(f"\n📈 Query Results:")
        print(f"Columns: {exec_result['columns']}")
        print(f"Row Count: {exec_result['row_count']}")
        print("✅ RLS user workflow completed successfully!")
        
        # Display ALL rows for complete comparison
        if exec_result['rows']:
            print(f"\n📊 COMPLETE Data Results (all {exec_result['row_count']} rows):")
            for i, row in enumerate(exec_result['rows']):
                print(f"Row {i+1}: {row}")
    else:
        print(f"❌ Query execution failed: {exec_result['error_message']}")
else:
    print(f"❌ Workflow failed: {rls_result.get('error', 'Unknown error')}")

print("\n" + "=" * 60)

# 📊 Results Comparison
print("📊 DETAILED WORKFLOW COMPARISON SUMMARY")
print("=" * 60)

if 'regular_result' in locals() and 'rls_result' in locals():
    if regular_result['workflow_complete'] and rls_result['workflow_complete']:
        reg_rows = regular_result['execution_result'].get('row_count', 0)
        rls_rows = rls_result['execution_result'].get('row_count', 0)
        
        print(f"🔢 DATA COMPARISON:")
        print(f"  Regular User Results: {reg_rows} rows")
        print(f"  RLS User Results: {rls_rows} rows")
        print(f"  Difference: {reg_rows - rls_rows} rows filtered by RLS")
        
        print(f"\n🔍 DAX QUERY COMPARISON:")
        print(f"  Regular Query Length: {len(regular_result['dax_query'])} characters")
        print(f"  RLS Query Length: {len(rls_result['dax_query'])} characters")
        print(f"  RLS Filter Detected: {'FILTER(' in rls_result['dax_query'] or 'Canada' in rls_result['dax_query']}")
        
        if regular_result['execution_result']['status'] == 'success' and rls_result['execution_result']['status'] == 'success':
            reg_data = regular_result['execution_result']['rows']
            rls_data = rls_result['execution_result']['rows']
            
            print(f"\n📊 DATA VALUES COMPARISON:")
            if reg_data and rls_data:
                print("  Regular User - Sample Values:")
                for i, row in enumerate(reg_data[:2]):
                    print(f"    Year {row[0]}: ${row[1]:,.2f}")
                
                print("  RLS User - Sample Values:")
                for i, row in enumerate(rls_data[:2]):
                    print(f"    Year {row[0]}: ${row[1]:,.2f}")
        
        print(f"\n🎯 Agentic Architecture Benefits:")
        print(f"✅ Plugin-based modularity")
        print(f"✅ Consistent workflow orchestration")
        print(f"✅ Automatic RLS context handling")
        print(f"✅ Real LLM integration")
        print(f"✅ Error handling and logging")
        print(f"✅ Testable and maintainable code")
        
        print(f"\n🏗️ Architecture Complete!")
        print(f"4/4 plugins implemented and working:")
        print(f"  👤 UserContextPlugin")  
        print(f"  🧠 DAXGenerationPlugin")
        print(f"  🔐 AuthenticationPlugin")
        print(f"  📊 QueryExecutionPlugin")
    else:
        print("⚠️ One or both workflows failed - check error messages above")
else:
    print("⚠️ Results not available for comparison")

🧪 TESTING COMPLETE AGENTIC WORKFLOW - RLS USER
🤖 Complete DAX Copilot Agent Processing: 'Show Sales by Year'
👤 User Type: rls

📋 Step 1: Determining user context...
✅ User context determined

🧠 Step 2: Generating DAX query with LLM...
✅ DAX query generated with LLM
Generated DAX: EVALUATE
SUMMARIZE(
    FILTER(
        FactInternetSales,
        RELATED(DimSalesTerritory[Sales T...

🔐 Step 3: Acquiring Power BI access token...
✅ Access token acquired

📊 Step 4: Executing DAX query against Power BI...
✅ DAX query executed

📊 RLS USER RESULTS:
Question: Show Sales by Year
User Type: rls
RLS Applied: Yes
RLS Filter Applied: DimSalesTerritory[Sales Territory Country] = "Canada"

📜 COMPLETE Generated DAX Query (with RLS filter):
EVALUATE
SUMMARIZE(
    FILTER(
        FactInternetSales,
        RELATED(DimSalesTerritory[Sales Territory Country]) = "Canada"
    ),
    DimDate[CalendarYear],
    "Sales", [Sales]
)

📈 Query Results:
Columns: ['DimDate[CalendarYear]', '[Sales]']
Row Count: 4
✅ 